## Define Operator Algebra

In [26]:
using LinearAlgebra

struct Operator  # Linear Matrix Operators from Matrices to Matrices (and the operator adjoint)
    op
    adj
    sym
end

## Operators
ℒ(A::Matrix)  = Operator(X->A*X   , X->A'*X, "ℒ$(size(A))"  )   # left multiply by A (X → AX)
ℛ(A::Matrix)  = Operator(X->X*A   , X->X*A', "ℛ$(size(A))")     # right multiply by A (X → XA)
ℋ(A::Matrix)  = Operator(X->X.*A  , X->X.*A, "ℋ$(size(A))")    # Hadamard product (elementwise product)
ℐ()  =          Operator(X->X      ,    X->X,    "I")     # identity operator
𝒪()  =           Operator(X->zero(X) , X->zero(X),"𝒪")# zero operator

import Base:  zero, show, adjoint, *, \, ∘, +, -
show(io::IO, M::Operator) = print(io, M.sym)  # pretty printing
zero(::Any) = 𝒪() # Let's make any undefined zero the 𝒪 operator
adjoint(A::Operator) = Operator(A.adj, A.op,  "("*A.sym*")'")
adjoint(B::Bidiagonal) = Bidiagonal(adjoint.(B.dv),adjoint.(B.ev),(B.uplo == 'U') ? :L : :U) # lower to upper
-(A::Operator) = Operator(X->-A.op(X), X->-A.adj(X),"-"*A.sym)
-(::typeof(𝒪), X::Matrix) = -X # 𝒪 - X should be -X
*(A::Operator, X::Matrix) = A.op(X)
\(ℐ::typeof(ℐ()), A::Matrix) = A
∘(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)
# We need [A;B]*C to somehow magically be [AC;BC]
*(M::Adjoint{Operator, Matrix{Operator}},v::Array) = M .* [v]
+(A::Array,x::Number)=A.+x

+ (generic function with 192 methods)

## Example

In [2]:
# Basic Test
B = [ 1 2; 3 4]
M = [10 1;1 10]
C = [ 2 5;4 6]
ℒ(M)

ℒ(2, 2)

In [5]:
typeof(ℒ(M))

Operator

In [3]:
ℒ(M) * [ 1 0 ;0 1]

2×2 Matrix{Int64}:
 10   1
  1  10

In [4]:
ℒ(M) * B 

2×2 Matrix{Int64}:
 13  24
 31  42

In [13]:
ℒ(M).op(B)

2×2 Matrix{Int64}:
 13  24
 31  42

In [6]:
M * B

2×2 Matrix{Int64}:
 13  24
 31  42

In [7]:
ℛ(M) * B 

2×2 Matrix{Int64}:
 12  21
 34  43

In [8]:
B * M # right multiply by M

2×2 Matrix{Int64}:
 12  21
 34  43

In [10]:
[ℋ(M) * B M .* B]

2×4 Matrix{Int64}:
 10   2  10   2
  3  40   3  40

In [14]:
# tr( B'*(ℒ(M)*C) ), tr( (ℒ(M)'*B) * C)     # This is not correct
tr( B'*(ℒ(M)*C) ), tr( (ℒ(M)'*B)' * C)    # <B,ℒC>=<ℒ'B,C>

(522, 522)

In [15]:
tr(B' * ℒ(M).op(C)), tr( ℒ(M).adj(B)' * C)

(522, 522)

In [16]:
B = [ 1 2; 3 4]
M = Bidiagonal( [ℐ(),ℐ(),ℐ()] , [ℒ(B),ℒ(B)], :L)
display(Matrix(M))

3×3 Matrix{Operator}:
 I        𝒪        𝒪
 ℒ(2, 2)  I        𝒪
 𝒪        ℒ(2, 2)  I

In [20]:
display(Matrix(M'))

3×3 Matrix{Operator}:
 (I)'  (ℒ(2, 2))'  𝒪
 𝒪     (I)'        (ℒ(2, 2))'
 𝒪     𝒪           (I)'

In [27]:
M'

3×3 Bidiagonal{Operator, Vector{Operator}}:
 (I)'  (ℒ(2, 2))'  ⋅
 ⋅     (I)'        (ℒ(2, 2))'
 ⋅     ⋅           (I)'

In [21]:
b = [ rand(2,2) for i=1:3]
x = M'\b
display(M'*x .- b)

3-element Vector{Matrix{Float64}}:
 [2.220446049250313e-16 -7.771561172376096e-16; -6.661338147750939e-16 4.440892098500626e-16]
 [-1.1102230246251565e-16 -1.1102230246251565e-16; 1.1102230246251565e-16 -2.220446049250313e-16]
 [0.0 0.0; 0.0 0.0]

In [28]:
M = Bidiagonal( [ℐ(),ℐ(),ℐ()] , [ℒ(B),ℒ(B)], :L)
display(Matrix(M))

b = [ rand(2,2) for i=1:3]
display(b)
x = M'\b
display(M'*x .- b)
display(Matrix(M'))

x = M\b
M*x .- b

3×3 Matrix{Operator}:
 I        𝒪        𝒪
 ℒ(2, 2)  I        𝒪
 𝒪        ℒ(2, 2)  I

3-element Vector{Matrix{Float64}}:
 [0.7719977794912496 0.7582154621428091; 0.007927160750158646 0.4328416654049634]
 [0.5137383247243174 0.19127429104242055; 0.06321595194544294 0.48846760848149684]
 [0.3527602092549482 0.6681560405771582; 0.1458331823967195 0.031130922320351195]

3-element Vector{Matrix{Float64}}:
 [4.440892098500626e-16 -3.3306690738754696e-16; -1.1102230246251565e-16 -4.440892098500626e-16]
 [0.0 0.0; 0.0 0.0]
 [0.0 0.0; 0.0 0.0]

3×3 Matrix{Operator}:
 (I)'  (ℒ(2, 2))'  𝒪
 𝒪     (I)'        (ℒ(2, 2))'
 𝒪     𝒪           (I)'

3-element Vector{Matrix{Float64}}:
 [0.0 0.0; 0.0 0.0]
 [0.0 0.0; 0.0 -1.1102230246251565e-16]
 [2.220446049250313e-16 -7.771561172376096e-16; 1.1102230246251565e-16 -3.3306690738754696e-16]

## Simple neural net

In [27]:
using OffsetArrays

h(x) =   exp(-x) # sample activation function
h′(x) = -exp(-x)

function neural_net(params,X₀;h=h,h′= h′)
    T = Matrix{Float64}
    N = length(params)
    X = OffsetArray(Vector{T}(undef,N+1),0:N)   
    Δ = Vector{T}(undef, N)
    X[0] = X₀
    W = first.(params)
    B = last.(params)
    
    for i=1:N         
          X[i] =  h.(W[i]*X[i-1] .+ B[i])
          Δ[i] =  h′.(W[i]*X[i-1] .+ B[i])        
    end 
    X,Δ
end

neural_net (generic function with 1 method)

## Initialization

In [28]:
n = [5,4,3,1]  ## this contains [n₀...n_N]
k = 10 # batchsize
N = length(n)-1 #should be positive
init(sizes...) = 0.01randn(sizes...)
Ws_and_bs =[ [init(n[i+1],n[i]) , init(n[i+1])]  for i=1:N] # The second part of the pair is a vector here
X₀ = init(n[1],k)
y  =  init(n[end],k); #  y is what we will compare X_N against
X,δ = neural_net(Ws_and_bs,X₀) # This has all the X's and δ's

𝓁(x,y) = sum(abs2,x-y)/2 #loss
𝓁′(x,y) = x.-y;

X,δ = neural_net(Ws_and_bs,X₀) # Run the neural net

([[-0.0005010996245781859 -0.014877576975130266 … 0.0046236491232974205 0.004129015847325036; -0.01345592108039518 0.012053612096829414 … 0.0017929618887288737 0.006554540032162239; … ; -0.006979695721620757 -0.017891939753068114 … 0.0035010505590273846 0.022044494068141504; 0.007944585875558336 0.004301709790477128 … -0.0005096040059599075 0.020695008562328154], [0.9954442737745843 0.9954531635303702 … 0.995279051628559 0.9949801237625622; 1.00270422344686 1.002968480382611 … 1.0028747037784551 1.0026680193139619; 0.9909793667045843 0.9912676079542091 … 0.9908735452849461 0.9900563265409489; 0.9866851263196393 0.986589198835786 … 0.9867984156393413 0.9867248120143067], [1.0282231832434876 1.0282231192380096 … 1.028226385562727 1.0282256557621914; 1.0009933116339107 1.0009927261363074 … 1.000991403353072 1.0009937771425184; 0.9789390724760894 0.9789322251044572 … 0.9789429578605084 0.97896169236286], [0.9958035197714701 0.9958036058611739 … 0.9958034900686087 0.9958032500727938]], [[-0

In [14]:
# params: `W_i` and `b_i`s: x_{i+1} <- Wi*x_i .+ b_i
Ws_and_bs =[ [init(n[i+1],n[i]) , init(n[i+1],k)]  for i=1:N] # The second part of the pair is a vector here
X₀ = init(n[1],k)
y  =  init(n[end],k); #  y is what we will compare X_N against
Ws_and_bs

3-element Vector{Vector{Matrix{Float64}}}:
 [[-0.012143092646801593 0.003085823585681731 … -0.003167963113614788 0.009596009661363319; -0.004337762112176135 0.01036608887453694 … 0.005581008479239419 -0.0007020755509605791; 0.014853864599875201 0.021559341643004108 … -0.008651655365812966 0.006115819359156278; -0.026280537238715197 0.0071992297108035485 … 0.006212874031915643 0.003297947312389446], [0.010124943618193329 -0.00025042501816379065 … 0.00863297763320263 -0.004354179124755006; -0.001075709695366005 0.0051588316697859715 … -0.006054310659378832 0.011078063256761242; -0.01909868044021105 -0.005730351022052706 … -0.015882019724896477 -0.00048084291354487694; 0.0049544571916539226 -0.0023089690395337783 … -0.0027878323531061178 0.0025997643434722373]]
 [[0.003001198161846007 0.008244343773183107 0.014301576004232304 -0.0005840931884569898; -0.021669175196715865 0.004129064636437034 -0.012704334302578233 -0.002263452940631432; -0.010408180508868421 0.016616016983090703 0.01094143

## Backward diff a neural net with operators

In [29]:
#X,δ = neural_net(Ws_and_bs,X₀) # This has all the X's and δ's

## The diagonal matrix
M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i])] for i=1:N])

## The lower triangular matrix (I-L)
ImL = Bidiagonal([ℐ() for i in 1:N], -[ℋ(δ[i]) ∘ ℒ(Ws_and_bs[i][1]) for i=2:N] , :L)

## gradient of the loss function
g = [ fill(𝒪,N-1) ; [𝓁′(X[N],y)] ] 
∇J = M' * (ImL' \ g)

3-element Vector{Matrix{Matrix{Float64}}}:
 [[4.311753896475572e-6 3.0879952678228063e-6 … 3.515014708720211e-6 -8.028878050327818e-6; 2.6901000843524824e-6 1.9259980941978677e-6 … 2.1931551224858425e-6 -5.011189905742933e-6; 5.656262352679766e-6 4.0518337288900184e-6 … 4.618929047919463e-6 -1.0527968790411956e-5; 4.5189143049100303e-7 3.237667963346485e-7 … 3.6860901007382726e-7 -8.422125820929734e-7]; [-0.00015769182485293065 -0.0001597522217276599 … -0.0001557276386337012 -0.00015991482178697488; -9.838562123012896e-5 -9.969687048248743e-5 … -9.719295131806385e-5 -9.98145526850621e-5; -0.00020685713801172609 -0.00020961893092162468 … -0.00020429284901355037 -0.0002096759653621855; -1.6537544878783224e-5 -1.6752516021956517e-5 … -1.633644339287579e-5 -1.677743897271121e-5];;]
 [[-0.11795327594624452 -0.11884768452843669 -0.11743756161637763 -0.11692890989976551; -0.07718864538141758 -0.07777394648721248 -0.07685116178438253 -0.07651830000559691; 0.12990940635813028 0.1308944751995437

In [34]:
∇J[1][2]

4×10 Matrix{Float64}:
 -0.000157692  -0.000159752  -0.000155974  …  -0.000155728  -0.000159915
 -9.83856e-5   -9.96969e-5   -9.73362e-5      -9.7193e-5    -9.98146e-5
 -0.000206857  -0.000209619  -0.000204656     -0.000204293  -0.000209676
 -1.65375e-5   -1.67525e-5   -1.63621e-5      -1.63364e-5   -1.67774e-5

In [30]:
#∇Jfd is the gradient calculated with finite differences method
∇Jfd = Ws_and_bs*0
ϵ = Ws_and_bs*0
𝜀 = .0001
for i=1:length(Ws_and_bs), wb=1:2
    for j=1:length(ϵ[i][wb])
        ϵ[i][wb][j] = 𝜀
        ∇Jfd[i][wb][j] = (𝓁(neural_net(Ws_and_bs+ϵ,X₀)[1][N],y).-𝓁(neural_net(Ws_and_bs-ϵ,X₀)[1][N],y))/2𝜀
        ϵ[i][wb][j] = .0
  end
 end

In [21]:
flatten(J) = vcat((x->x[:]).(vcat(J...))...)

flatten (generic function with 1 method)

In [22]:
norm(flatten(∇J)-flatten(∇Jfd))

1.0741248302637355e-7